In [2]:
from manim import *
from numpy import linalg as npl
import math
import matplotlib.pyplot as plt
import jupyter_capture_output
from scipy.interpolate import make_interp_spline, BSpline
from multiprocessing import Process

video_scene = " -v WARNING --progress_bar None --disable_caching heat_Scene"
image_scene = f" -v WARNING --progress_bar None --disable_caching -r {2*427},{2*240}  -s heat_Scene"

Manim Community v0.16.0.post0

Jupyter Capture Output v0.0.8


In [59]:
%%capture_video --path "animations/heat_equation/heat_equation.mp4"
%%manim -qh --fps 60 $video_scene


### Datenerzeugung ###
A22_Heat_data = np.loadtxt("external_data/A22_Waermeleitungsgleichung_data.csv", delimiter = ",", skiprows = 0)

x_array = A22_Heat_data[0,:]
heat_iter_list = []

# Erzeugung des Iter-Arrays
for i in range(len(x_array)):
    heat_iter_list.append(iter(A22_Heat_data[1::10,i]))
### Datenerzeugung ###


class heat_Scene(Scene):
    def construct(self):
        CVC = Text('CVC', font_size = 12, weight = BOLD, color = WHITE, font = 'Latin Modern Sans').align_on_border(RIGHT + DOWN, buff = 0.2)
        self.add(CVC)


        # Header
        text_heat = Tex(r"Wärmeleitungsgleichung: $\partial_t T(x,t)=D\cdot\partial_x^2 T(x, t)$", font_size = 48).align_on_border(UP + LEFT, buff = 0.5).shift(RIGHT)
        self.add(text_heat)


        # Heat-Sources
        left_source = Circle(radius = 1.0, color = RED, fill_opacity = 0.5).move_to([-5.5, 0, -1])
        right_source = Circle(radius = 1.0, color = BLUE, fill_opacity = 0.5).move_to([5.5, 0, -1])
        self.add(left_source, right_source)


        # Heating Rode
        Rode = Rectangle(height = 0.75 + 0.02, width = 10 + 0.02, fill_color = WHITE, fill_opacity = 0.5, stroke_opacity = 1, stroke_color = WHITE)
        self.add(Rode)


        # Initial Heat-Elementes of the Rode
        T_line_list = []
        for i, heat_iter in enumerate(heat_iter_list):
            T = next(heat_iter)
            if T > 0:
                T_line = Line(start = [(-0.5 + x_array[i]) * 10, -0.75/2, 0], end = [(-0.5 + x_array[i]) * 10, 0.75/2, 0], color = RED, stroke_opacity = T)
            else:
                T_line = Line(start = [(-0.5 + x_array[i]) * 10, -0.75/2, 0], end = [(-0.5 + x_array[i]) * 10, 0.75/2, 0], color = BLUE, stroke_opacity = T)
            T_line_list.append(T_line)
            self.add(T_line_list[i])


        # Heat-Elements-Updater
        def T_updater(Rode):
            for i, heat_iter in enumerate(heat_iter_list):
                T = next(heat_iter)
                if T > 0:
                    T_line_list[i].become(Line(start = [(-0.5 + x_array[i]) * 10, -0.75/2, 0], end = [(-0.5 + x_array[i]) * 10, 0.75/2, 0], color = RED, stroke_opacity = T))
                else:
                    T_line_list[i].become(Line(start = [(-0.5 + x_array[i]) * 10, -0.75/2, 0], end = [(-0.5 + x_array[i]) * 10, 0.75/2, 0], color = BLUE, stroke_opacity = abs(T)))


        # Adding Dynamic
        time = ValueTracker(0)
        Rode.add_updater(T_updater)
        self.wait(0.5)
        self.play(time.animate.set_value(1), rate_func= linear, run_time = 19.5)


Output saved by overwring previous file at animations/heat_equation/heat_equation.mp4.
